In [2]:
# export
from collections.abc import MutableMapping
import json
import pprint
from typing import Callable, Tuple, Dict, Optional
import sqlite3

In [3]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

In [36]:
# export
class SQLQueue:
    def __init__(
        self,
        dbname = ":memory:",
        maxsize: Optional[int] = None,
        check_same_thread=False,
        fast=True,
        **kwargs,
    ):
        self.dbname = dbname
        self.conn = sqlite3.connect(
            self.dbname, check_same_thread=check_same_thread, **kwargs
        )
        self.maxsize = maxsize

        # self.conn.row_factory = sqlite3.Row

        # status 0: free, 1: locked, 2: done

        with self.conn as c:
            # int == bool in SQLite
            # will have rowid as primary key by default
            c.execute(
                """CREATE TABLE IF NOT EXISTS Queue 
                ( message TEXT NOT NULL,
                  task_id TEXT,
                  status INTEGER,
                  in_time INTEGER NOT NULL,
                  lock_time INTEGER,
                  out_time INTEGER )
                """
            )

            c.execute("CREATE INDEX TIdx ON Queue(task_id)")
            c.execute("CREATE INDEX SIdx ON Queue(status)")

            if fast:
                c.execute("PRAGMA journal_mode = 'WAL';")
                c.execute("PRAGMA temp_store = 2;")
                c.execute("PRAGMA synchronous = 1;")
                c.execute(f"PRAGMA cache_size = {-1 * 64_000};")

            if maxsize is not None:
                c.execute(
                    f"""
CREATE TRIGGER IF NOT EXISTS maxsize_control 
   BEFORE INSERT
   ON Queue
   WHEN (SELECT COUNT(*) FROM Queue WHERE status = 0) >= {self.maxsize}
BEGIN
    SELECT RAISE (ABORT,'Max queue length reached');
END;"""
                )

    def put(self, message: str, timeout: int = None) -> int:
        "Insert a new task"

        with self.conn as c:
            rid = c.execute(
                r"INSERT INTO Queue VALUES (:message, lower(hex(randomblob(16))), 0, strftime('%s','now'), NULL, NULL)",
                {"message": message},
            ).lastrowid

        return rid

    def pop(self) -> Dict[str, str]:

        # lastrowid not working as expected :(

        # this should happen all inside a transaction
        with self.conn as c:
            task = c.execute(
                """
            SELECT message, task_id FROM Queue
            WHERE rowid = (SELECT min(rowid) FROM Queue
                           WHERE status = 0 LIMIT 1)
            """
            ).fetchone()

            if task is None:
                return None

            c.execute(
                """
UPDATE Queue SET status = 1, lock_time = strftime('%s','now') WHERE task_id = :task_id""",
                {"task_id": task[1]},
            )

            return task

    def peek(self) -> Tuple:
        "Show next task to be popped."
        # order by should not be really needed
        value = self.conn.execute(
            "SELECT * FROM Queue WHERE status = 0 ORDER BY rowid LIMIT 1"
        ).fetchone()
        return value

    def get(self, task_id: str) -> Tuple:
        "Get a task by its `task_id`"

        return self.conn.execute(
            "SELECT * FROM Queue WHERE task_id = :task_id", {"task_id": task_id}
        ).fetchone()

    def done(self, task_id) -> int:
        """
        Mark task as done.
        If executed multiple times, `done_time` will be
        the last time this function is called.
        """

        with self.conn as c:
            x = c.execute(
                "UPDATE Queue SET status = 2,  out_time = strftime('%s','now') WHERE task_id = :task_id",
                {"task_id": task_id},
            ).lastrowid
        return x

    def qsize(self) -> int:
        return next(self.conn.execute("SELECT COUNT(*) FROM Queue WHERE status != 2"))[0]

    def empty(self) -> bool:
        value = self.conn.execute(
            """
SELECT (CASE SELECT COUNT(*) FROM Queue WHERE status = 0
-- empty
        WHEN 0 THEN 1
-- not empy
        ELSE 0
    END)
        """
        ).fetchone()

        return bool(value)

    def full(self) -> bool:
        return False

    def prune(self):
        with self.conn as c:
            c.execute("DELETE FROM Queue WHERE status = 2")

        self.conn.execute("VACUUM;")
        return

    def __repr__(self):
        return f"{type(self).__name__}(dbname={self.dbname!r}, items={pprint.pformat(self.conn.execute('SELECT * FROM Queue').fetchall())})"

    def close(self):
        self.conn.close()

In [37]:
q = SQLQueue(":memory:")

q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

4

In [38]:
q.pop()

('hello', '84da70f3532c23b083cdb3af500742e4')

In [39]:
print(q)

SQLQueue(dbname=':memory:', items=[('hello', '84da70f3532c23b083cdb3af500742e4', 1, 1612167131, 1612167131, None),
 ('world', '63e7f8baf48ca108e4fce1c03bff9b34', 0, 1612167131, None, None),
 ('foo', '07ded462bd85da00344d3b984906745e', 0, 1612167131, None, None),
 ('bar', '3af4ab0b37b3ffb96880868de6f413cf', 0, 1612167131, None, None)])


In [40]:
# pop remaining
for _ in range(3):
    q.pop()


assert q.pop() is None

In [41]:
print(q)

SQLQueue(dbname=':memory:', items=[('hello', '84da70f3532c23b083cdb3af500742e4', 1, 1612167131, 1612167131, None),
 ('world', '63e7f8baf48ca108e4fce1c03bff9b34', 1, 1612167131, 1612167131, None),
 ('foo', '07ded462bd85da00344d3b984906745e', 1, 1612167131, 1612167131, None),
 ('bar', '3af4ab0b37b3ffb96880868de6f413cf', 1, 1612167131, 1612167131, None)])


In [42]:
q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

8

In [43]:
message, task_id = q.pop()

assert message == "hello"

In [44]:
q.peek()

('world', '1c51f4550d1a207d3af0b4da84a3c88c', 0, 1612167131, None, None)

In [45]:
# next one that is free
assert q.peek()[0] == "world"

# status = 0 = free
assert q.peek()[2] == 0

In [46]:
message, task_id

('hello', '7872dd9caa283ea0519764fd31b9f241')

In [47]:
q.done(task_id)

8

In [48]:
q.get(task_id)

('hello',
 '7872dd9caa283ea0519764fd31b9f241',
 2,
 1612167131,
 1612167132,
 1612167133)

In [49]:
already_done = q.get(task_id)

# stauts = 2 = done
assert already_done[2] == 2

in_time = already_done[3]
lock_time = already_done[4]
done_time = already_done[5]

assert done_time >= lock_time >= in_time
print(
    f"Task {already_done[1]} took {done_time - lock_time} seconds to get done and was in the queue for {done_time - in_time} seconds"
)

Task 7872dd9caa283ea0519764fd31b9f241 took 1 seconds to get done and was in the queue for 2 seconds


In [50]:
print(q)

SQLQueue(dbname=':memory:', items=[('hello', '84da70f3532c23b083cdb3af500742e4', 1, 1612167131, 1612167131, None),
 ('world', '63e7f8baf48ca108e4fce1c03bff9b34', 1, 1612167131, 1612167131, None),
 ('foo', '07ded462bd85da00344d3b984906745e', 1, 1612167131, 1612167131, None),
 ('bar', '3af4ab0b37b3ffb96880868de6f413cf', 1, 1612167131, 1612167131, None),
 ('hello',
  '7872dd9caa283ea0519764fd31b9f241',
  2,
  1612167131,
  1612167132,
  1612167133),
 ('world', '1c51f4550d1a207d3af0b4da84a3c88c', 0, 1612167131, None, None),
 ('foo', 'cd893aa10729dbd5acad86b9ca41df7a', 0, 1612167131, None, None),
 ('bar', '95cba60c630c51fda03eb14d268179c3', 0, 1612167131, None, None)])


In [53]:
assert q.qsize() == 7

In [54]:
next_one_msg = q.peek()[0]
next_one_id = q.peek()[1]

message, task_id = q.pop()

assert message == next_one_msg
assert task_id == next_one_id

In [55]:
print(q)

SQLQueue(dbname=':memory:', items=[('hello', '84da70f3532c23b083cdb3af500742e4', 1, 1612167131, 1612167131, None),
 ('world', '63e7f8baf48ca108e4fce1c03bff9b34', 1, 1612167131, 1612167131, None),
 ('foo', '07ded462bd85da00344d3b984906745e', 1, 1612167131, 1612167131, None),
 ('bar', '3af4ab0b37b3ffb96880868de6f413cf', 1, 1612167131, 1612167131, None),
 ('hello',
  '7872dd9caa283ea0519764fd31b9f241',
  2,
  1612167131,
  1612167132,
  1612167133),
 ('world', '1c51f4550d1a207d3af0b4da84a3c88c', 1, 1612167131, 1612167147, None),
 ('foo', 'cd893aa10729dbd5acad86b9ca41df7a', 0, 1612167131, None, None),
 ('bar', '95cba60c630c51fda03eb14d268179c3', 0, 1612167131, None, None)])


In [56]:
q.prune()

In [57]:
print(q)

SQLQueue(dbname=':memory:', items=[('hello', '84da70f3532c23b083cdb3af500742e4', 1, 1612167131, 1612167131, None),
 ('world', '63e7f8baf48ca108e4fce1c03bff9b34', 1, 1612167131, 1612167131, None),
 ('foo', '07ded462bd85da00344d3b984906745e', 1, 1612167131, 1612167131, None),
 ('bar', '3af4ab0b37b3ffb96880868de6f413cf', 1, 1612167131, 1612167131, None),
 ('world', '1c51f4550d1a207d3af0b4da84a3c88c', 1, 1612167131, 1612167147, None),
 ('foo', 'cd893aa10729dbd5acad86b9ca41df7a', 0, 1612167131, None, None),
 ('bar', '95cba60c630c51fda03eb14d268179c3', 0, 1612167131, None, None)])


In [58]:
from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [59]:
q = SQLQueue(":memory:", maxsize=50)

In [60]:
for i in range(50):

    q.put(random_string(20))

In [61]:
assert q.qsize() == 50

In [63]:
try:
    q.put(random_string(20))
except sqlite3.IntegrityError: # max len reached
    print("test pass")

test pass


In [64]:
q.pop()

('tqqzfijeqmiohdzewzau', '97ae2bc5ab7940b5dd78f73c9ba42067')

In [65]:
q.put("hello")

51